In [1]:
%load_ext autoreload
%autoreload 2
from configparser import ConfigParser
import topic_modeler.modeler
import topic_modeler.clustering
import topic_modeler.modeling
import topic_modeler.newsifier
import importlib
importlib.reload(topic_modeler.clustering)
importlib.reload(topic_modeler.modeling)
importlib.reload(topic_modeler.newsifier)
importlib.reload(topic_modeler.modeler)
from topic_modeler.modeler import Modeler
import json
import pandas as pd

Using TensorFlow backend.


In [2]:
cfg = ConfigParser()
cfg.read('topic_modeler/settings.ini')
cluster_cfg = cfg['CLUSTERING']

In [ ]:
modeler = Modeler()

In [ ]:
clf = Pipeline([
            ('tfidf', modeler.article_vectorizer),
            ('clf', modeler.classifier),
        ])
cluster_names = json.loads(cluster_cfg['CLUSTER_NAMES'])

In [ ]:
def sorted_cluster_predictions(text):
    return [
             { 'topic' : k,  'prob' : v } for k, v in sorted(
                { cluster_names[i] : x 
                                for i, x in enumerate(
                                    clf.predict_proba(
                                        [text]
                                    )[0]
                                ) 
                }.items(),
                key=lambda item: item[1],
                reverse=True)
    ]

import re
import requests
def predict_url(url):
    print(url)
    text = text_from_html(requests.get(url).content)
    formatted_predictor(text)
    
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    return u" ".join(t.strip() for t in texts)


def formatted_predictor(text):
    ignore = ('Biology and Climate')
    general = ('General Climate')
    predictions = sorted_cluster_predictions(text)
    top = predictions.pop(0)
    while top['topic'] in ignore:
        top = predictions.pop(0)
    if (top['topic'] in general and top['prob'] < .32) or (top['prob'] < .1):
        print("This is probably not about climate.")
        print(f"(The best-scoring topic is {top['topic']} at {top['prob']:.2%})")
        return
    if top['prob'] > .9:
            print(f"This is almost definitely about {top['topic']} ({top['prob']:.2%}).")
    else:
        print(f"The best-scoring topic is {top['topic']} at {top['prob']:.2%}")
    top = predictions.pop(0)
    while top['prob'] > .05:
        if top['topic'] not in ignore:
            print(f"* {top['topic']} ({top['prob']:.2%})")
        top = predictions.pop(0)